In [1]:
import numpy as np #Linear algebra and mathematical operations
import pandas as pd #importing and loading data
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import time

In [2]:
iris_df = pd.read_csv("/content/drive/MyDrive/Data Science /Iris.csv")
iris_df = iris_df.sample(frac=1).reset_index(drop=True) # Shuffle

In [3]:
iris_df.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,58,4.9,2.4,3.3,1.0,Iris-versicolor
1,141,6.7,3.1,5.6,2.4,Iris-virginica
2,46,4.8,3.0,1.4,0.3,Iris-setosa
3,51,7.0,3.2,4.7,1.4,Iris-versicolor
4,104,6.3,2.9,5.6,1.8,Iris-virginica


In [4]:

X = iris_df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
X = np.array(X)
X[:5]


array([[4.9, 2.4, 3.3, 1. ],
       [6.7, 3.1, 5.6, 2.4],
       [4.8, 3. , 1.4, 0.3],
       [7. , 3.2, 4.7, 1.4],
       [6.3, 2.9, 5.6, 1.8]])

In [5]:
one_hot_encoder = OneHotEncoder(sparse=False)
Y = iris_df.Species
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))
Y[:5]

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

In [7]:
class ANN_Classification():

  
    def __init__ (self, hidden_Layer_Size = [100,], learning_Rate = 0.001, epochs = 10):
        self.hidden_Layer_Size = hidden_Layer_Size
        self.learning_Rate = learning_Rate
        self.epochs = epochs
       # self.activation_function = activation_function
        self.weights = None

    
    def sigmoid(self, x, derivative=False):
        if derivative:
            return np.multiply(x, 1-x)
        return 1/(1 + np.exp(-x))
        

    def softmax(self, x):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        return exps / np.sum(exps, axis=0)
            

    #function for forward propogation
    def forward_Prop(self, x, layers):
        activations, layer_input = [x], x
        for j in range(layers):
         # i = 
          #print("i = "+str(i))
          if j <= (layers-1):

            activation = self.sigmoid(np.dot(layer_input, self.weights[j].T))
            activations.append(activation)
            layer_input = np.append(1, activation)

          else :
            activation = self.softmax(np.dot(layer_input, self.weights[j].T))
            activations.append(activation)
            layer_input = np.append(1, activation)

        return activations


    def back_prop(self, y, activations, layers):
      outputFinal = activations[-1]
      error = np.matrix(y - outputFinal) 
      
      # Error after 1 cycle
      for j in range(layers, 0, -1):
        currActivation = activations[j]
       
        if(j > 1):
          # Append previous
          prevActivation = np.append(1, activations[j-1])
        else:
          # First hidden layer
          prevActivation = activations[0]
       
        delta = np.multiply(error, self.sigmoid(currActivation, derivative = True))
        self.weights[j-1] += self.learning_Rate * np.multiply(delta.T, prevActivation)
         
        wc = np.delete(self.weights[j-1], [0], axis=1)
        error = np.dot(delta, wc) #current layer error
       
      return self.weights

    
    def initialize_Weight(self, layers):
      layer, self.weights = len(layers), []
      #for loop to intialize the weight randomly
      for i in range(1, layer):
        #assigning random weights
        w = [[np.random.uniform(-1, 1) for j in range(layers[i-1] + 1)]for k in range(layers[i])]
        self.weights.append(np.matrix(w))
    
      return self.weights
    

    #train function
    def train(self, X, y):
        layers_weights = len(self.weights)
        
        for i in range(len(self.X)):
          x, y = self.X[i], self.y[i]
          x = np.matrix(np.append(1, x))
          
          activations = self.forward_Prop(x, layers_weights)
          self.weights = self.back_prop(y, activations, layers_weights)
          
        return self.weights


    def fit(self, X, y):
        intiate_time = time.perf_counter()
        self.X = X
        self.y = y
        hidden_Layers = len(self.hidden_Layer_Size) - 1
        self.weights = self.initialize_Weight(self.hidden_Layer_Size)

        for epoch in range(1, self.epochs+1):
          epoch_intiate_time = time.perf_counter()
          weights = self.train(self.X, self.y)

          epoch_closing_time = time.perf_counter()
          closing_time = time.perf_counter()
          
          epoch_total_time = epoch_closing_time - epoch_intiate_time
          total_time = closing_time - intiate_time
          print ("Epoch : {}".format(epoch))
          print ("Elapsed Time : {}".format(total_time))
          print ("Step Time : {}\n\n".format(epoch_total_time))
          
        return self.weights

        
    def Predict(self, X):
        result = []
        for i in range(len(X)):
          x = X[i]
          #print(str(i) + " " + str(len(X)))
          layers = len(self.weights)
          item = np.append(1, x)

          # Forward prop.
          activations = self.forward_Prop(item, layers)
          
          Final_output = activations[-1].A1
          index = self.FindMaxActivation(Final_output)
          
          predicted = [0 for j in range(len(Final_output))]
          predicted[index] = 1 
      
          
          result.append(predicted)
        
        return result
      
    def FindMaxActivation(self, output):
        m, index = output[0], 0
        for i in range(1, len(output)):
          if(output[i] > m):
            m, index = output[i], i
        
        return index

In [8]:
first = len(X[0]) # no. of features
output = len(Y[0]) # no. of classes

# Define hyperparameters to search over
hidden_layer_sizes = [[first, 50, output], [first, 100, output], [first, 50, 50, output], [first, 100, 100, output]]
learning_rates = [0.001, 0.01, 0.1]
epochs = [10, 20, 30]

best_score = 0
best_params = {}

# Loop over all possible hyperparameter combinations
for hidden_size in hidden_layer_sizes:
    for lr in learning_rates:
        for epoch in epochs:
            
            model = ANN_Classification(hidden_Layer_Size=hidden_size, learning_Rate=lr, epochs=epoch)
            model.fit(X_train, Y_train)
            
            y_pred = model.Predict(X_test)
            score = accuracy_score(Y_test, y_pred)
            
            if score > best_score:
                best_score = score
                best_params = {'hidden_Layer_Size': hidden_size, 'learning_Rate': lr, 'epochs': epoch}

final_model = ANN_Classification(**best_params)
final_model.fit(np.concatenate([X_train, X_test]), np.concatenate([Y_train, Y_test]))


y_pred = final_model.Predict(X_test)
final_score = accuracy_score(Y_test, y_pred)
print("Best Parameters : {}".format(best_params))
print("Final Score = {}".format(final_score))

Epoch : 1
Elapsed Time : 0.041006984999995666
Step Time : 0.03852846199998794


Epoch : 2
Elapsed Time : 0.07272108099999741
Step Time : 0.031044368000010536


Epoch : 3
Elapsed Time : 0.10816448999997874
Step Time : 0.03522184199999856


Epoch : 4
Elapsed Time : 0.13954895599999873
Step Time : 0.03112166400001115


Epoch : 5
Elapsed Time : 0.17585405399998422
Step Time : 0.03607402599999432


Epoch : 6
Elapsed Time : 0.21403222199998595
Step Time : 0.03659417800000142


Epoch : 7
Elapsed Time : 0.268119243000001
Step Time : 0.05385949799998002


Epoch : 8
Elapsed Time : 0.3015590469999836
Step Time : 0.03247992999999383


Epoch : 9
Elapsed Time : 0.34527563599999667
Step Time : 0.04349107799998819


Epoch : 10
Elapsed Time : 0.4154734949999863
Step Time : 0.0672652199999959


Epoch : 1
Elapsed Time : 0.06752046900001574
Step Time : 0.06539371099998448


Epoch : 2
Elapsed Time : 0.12468891100002111
Step Time : 0.05572123300001408


Epoch : 3
Elapsed Time : 0.18113271000001419
Step Time

In [9]:
first = len(X[0]) # no. of features
output = len(Y[0]) # no. of classes

layers = [first, 5, 10, output] # no. of nodes 
L, E = 0.15, 200
#calling neural network function
weights = ANN_Classification(hidden_Layer_Size = layers, epochs=E, learning_Rate = L)

In [10]:
ans = weights.fit(X_train, Y_train)

Epoch : 1
Elapsed Time : 0.0742244909999954
Step Time : 0.0734888300000307


Epoch : 2
Elapsed Time : 0.11941034000000172
Step Time : 0.044918876999986423


Epoch : 3
Elapsed Time : 0.1700959629999943
Step Time : 0.04956949900002883


Epoch : 4
Elapsed Time : 0.21479636700001947
Step Time : 0.04446035699999129


Epoch : 5
Elapsed Time : 0.25764712599999484
Step Time : 0.04123146500000985


Epoch : 6
Elapsed Time : 0.31644749399998773
Step Time : 0.05857795599996507


Epoch : 7
Elapsed Time : 0.35864506900003335
Step Time : 0.04062237900001264


Epoch : 8
Elapsed Time : 0.41287958000003755
Step Time : 0.05278788399999712


Epoch : 9
Elapsed Time : 0.46338943599999993
Step Time : 0.05026770199998509


Epoch : 10
Elapsed Time : 0.5139232590000233
Step Time : 0.049437063999960174


Epoch : 11
Elapsed Time : 0.5621219079999946
Step Time : 0.04798470200000793


Epoch : 12
Elapsed Time : 0.6114461740000365
Step Time : 0.04906449400004931


Epoch : 13
Elapsed Time : 0.6810350490000019
Step Tim

In [11]:
y_pred = weights.Predict(X_test)
accuracy_score(Y_test, y_pred)

0.9130434782608695

# **Using Keras**

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [13]:
# Build the model

model = Sequential()

model.add(Dense(5, input_shape=(4,), activation='sigmoid', name='h1'))
model.add(Dense(10, activation='sigmoid', name='h2'))
model.add(Dense(3, activation='softmax', name='output'))

# Adam optimizer with learning rate of 0.001
optimizer = Adam(lr=0.15)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
model.fit(X_train, Y_train, epochs=200)

Epoch 1/200
4/4 [==============================] - 1s 6ms/step - loss: 1.3865 - accuracy: 0.2835
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 1.0916 - accuracy: 0.3465
Epoch 3/200
4/4 [==============================] - 0s 4ms/step - loss: 0.9641 - accuracy: 0.4173
Epoch 4/200
4/4 [==============================] - 0s 7ms/step - loss: 0.7229 - accuracy: 0.6457
Epoch 5/200
4/4 [==============================] - 0s 5ms/step - loss: 0.5159 - accuracy: 0.7953
Epoch 6/200
4/4 [==============================] - 0s 4ms/step - loss: 0.4027 - accuracy: 0.7953
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 0.3348 - accuracy: 0.9685
Epoch 8/200
4/4 [==============================] - 0s 8ms/step - loss: 0.2703 - accuracy: 0.9528
Epoch 9/200
4/4 [==============================] - 0s 4ms/step - loss: 0.1919 - accuracy: 0.9528
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 0.2154 - accuracy: 0.9449
Epoch 11/200
4/4 [===========

In [15]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 378ms/step - loss: 0.2363 - accuracy: 0.9130


[0.2363048493862152, 0.9130434989929199]